In [38]:
from sklearn import decomposition
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from PIL import Image
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.metrics import classification_report

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras import Sequential
from keras.applications.vgg16 import preprocess_input
import h5py as h5py
from keras.models import Model
from keras.layers import Activation, Flatten, Dense, GlobalAveragePooling2D

In [2]:
N = 150
w,h = 200, 200
size = (w, h, 3)
female = glob('females/*')
male = glob('males/*')


female_list = np.random.choice(female, N, replace=False)
male_list = np.random.choice(male, N, replace=False)

In [3]:
def pic_preprocess (image_file, width, height):
    
    img = Image.open(image_file)
    post = img.resize((width, height), Image.NEAREST)
    
    pix = np.array(post)
    return pix.flatten()
#     return post.getcolors(height*width)




In [4]:
raw_data = [(pic_preprocess(f, w, h), 0) for f in female_list] + [(pic_preprocess(m, w, h),1) for m in male_list]


labels = np.array([datapoint[1] for datapoint in raw_data])
data = np.array([datapoint[0] for datapoint in raw_data])
norm_data = np.array([datapoint[0] - np.mean(datapoint[0]) for datapoint in raw_data])

In [14]:
model = VGG16(weights='imagenet', include_top=False)

preds = []
for pic in female[0:20]:
    img_path = pic
    img = image.load_img(img_path, target_size=(224, 224))

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = model.predict(x)
    preds.append(features)

In [40]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3))

# add a global spatial average pooling layer
x = base_model.output
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer 
top_model = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=top_model)